In [1]:
import os
import io
import imageio
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers



In [2]:
DATASET_NAME = "hockey_fight_videos"
BATCH_SIZE = 64
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (36, 28, 28, 3)
NUM_CLASSES = 2
LABELS = ["no","fi"]

# OPTIMIZER
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.00001

# TRAINING
EPOCHS = 100

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

## DATASET

In [3]:
import os
import random
from sklearn.model_selection import train_test_split

# veri kümenizi yükleyin
data_path = "C:/Users/Emirhan/Downloads/data"
classes = os.listdir(data_path)

videos = []
labels = []
train_files = []
val_files = []
for cls in classes:
    cls_path = os.path.join(data_path, cls)
    files = os.listdir(cls_path)
    for video in os.listdir(cls_path):
        video_path = os.path.join(cls_path, video)
        videos.append(video_path)
        label = os.path.basename(video_path).split('_')[0][0:2]
        if label not in LABELS:
            continue
        labels.append(label)



In [4]:
labels = np.array(labels)
videos = np.array(videos)

In [5]:
from sklearn.preprocessing import OneHotEncoder

# Örnek veri etiketleri

# One-Hot Encoding
encoder = OneHotEncoder(sparse=False)
labels = encoder.fit_transform(np.array(labels).reshape(-1, 1))

# Kodlanmış etiketleri kontrol etme
print(labels)


[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


c:\Users\Emirhan\anaconda3\envs\tfenv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
import cv2
import numpy as np

# Videoların yollarını içeren bir liste oluşturma
video_paths = videos

# En kısa video uzunluğunu belirleme
min_frame_count = float('inf')

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < min_frame_count:
        min_frame_count = frame_count

# Tüm videolardan aynı sayıda frame almak için döngü
frames = []

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    selected_frames = []
    for i in range(min_frame_count):
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (28,28))
            selected_frames.append(frame)
    frames.append(np.array(selected_frames))

# Tüm videolardaki ortak frame sayısına sahip bir matris oluşturma
video_array = np.stack(frames)

# Video dizisinin şeklini yazdırma
print(video_array.shape)

(2000, 36, 28, 28, 3)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(video_array, labels, test_size=0.25)

In [8]:
X_train = X_train/255
X_test = X_test/255

In [9]:
len(y_train)

1500

In [10]:
len(X_test)

500

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [12]:
len(X_val)

150

In [13]:
X_val = X_val/255

In [14]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


trainloader = prepare_dataloader(X_train, y_train, "train")
validloader = prepare_dataloader(X_val, y_val, "valid")
testloader = prepare_dataloader(X_test, y_test, "test")

In [15]:
testloader

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 36, 28, 28, 3, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [16]:
# TUBELET EMBEDDING

class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches


In [17]:
# POSITIONAL 

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [18]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [19]:
def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=['accuracy'
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)


    return model


model = run_experiment()

Epoch 1/100
22/22 [==============================] - 14s 136ms/step - loss: 0.7844 - accuracy: 0.4837 - val_loss: 0.7503 - val_accuracy: 0.4933
Epoch 2/100
22/22 [==============================] - 2s 84ms/step - loss: 0.6862 - accuracy: 0.5630 - val_loss: 0.7927 - val_accuracy: 0.4933
Epoch 3/100
22/22 [==============================] - 2s 85ms/step - loss: 0.6879 - accuracy: 0.5526 - val_loss: 0.7714 - val_accuracy: 0.4933
Epoch 4/100
22/22 [==============================] - 2s 86ms/step - loss: 0.6665 - accuracy: 0.6059 - val_loss: 0.9102 - val_accuracy: 0.4933
Epoch 5/100
22/22 [==============================] - 2s 86ms/step - loss: 0.6334 - accuracy: 0.6407 - val_loss: 1.2121 - val_accuracy: 0.4933
Epoch 6/100
22/22 [==============================] - 2s 83ms/step - loss: 0.6023 - accuracy: 0.6519 - val_loss: 1.3483 - val_accuracy: 0.4933
Epoch 7/100
22/22 [==============================] - 2s 84ms/step - loss: 0.6349 - accuracy: 0.6356 - val_loss: 1.4950 - val_accuracy: 0.4933
Epoc

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 36, 28, 28,  0           []                               
                                 3)]                                                              
                                                                                                  
 tubelet_embedding (TubeletEmbe  (None, 36, 128)     196736      ['input_1[0][0]']                
 dding)                                                                                           
                                                                                                  
 positional_encoder (Positional  (None, 36, 128)     4608        ['tubelet_embedding[0][0]']      
 Encoder)                                                                                     

In [21]:
import numpy as np

# test verilerinizi hazırlayın
x_test = X_test

# tahminleri yapın
y_pred = model.predict(x_test)

# tahmin sonuçlarını kontrol edin
print(y_pred)  # (num_samples, num_classes)


16/16 [==============================] - 1s 23ms/step
[[1.00000000e+00 3.09020791e-08]
 [1.17269182e-03 9.98827279e-01]
 [8.01960632e-05 9.99919772e-01]
 [9.99992847e-01 7.14621319e-06]
 [2.34825988e-04 9.99765217e-01]
 [9.99265730e-01 7.34223227e-04]
 [1.00000000e+00 1.88673788e-08]
 [1.42474956e-08 1.00000000e+00]
 [3.90096358e-03 9.96098995e-01]
 [1.44327732e-08 1.00000000e+00]
 [9.99989390e-01 1.06207917e-05]
 [1.76939974e-08 1.00000000e+00]
 [1.79377399e-04 9.99820650e-01]
 [8.89930248e-01 1.10069707e-01]
 [2.57459465e-06 9.99997377e-01]
 [6.74042226e-08 9.99999881e-01]
 [1.00000000e+00 1.61870783e-08]
 [9.20372367e-01 7.96276107e-02]
 [1.89963691e-02 9.81003642e-01]
 [9.59989488e-01 4.00104932e-02]
 [5.75215563e-05 9.99942422e-01]
 [1.33834615e-01 8.66165340e-01]
 [1.68547700e-08 1.00000000e+00]
 [3.60223638e-08 1.00000000e+00]
 [9.99988675e-01 1.13258393e-05]
 [8.65144187e-08 9.99999881e-01]
 [1.90496841e-03 9.98094976e-01]
 [2.20354614e-05 9.99977946e-01]
 [4.36792493e-01 5.632

In [22]:
predictions = y_pred > 0.5

In [23]:
predictions

array([[ True, False],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [ True, False],
       [ True, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [False,  True],
       [ True, False],
       [ True, False],
       [False,  True],
       [ True, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [ True, False],
       [False,  True],
       [False,  True],
       [False,  True],
       [False,  True],
       [ True, False],
       [False,  True],
       [ True, False],
       [ True, False],
       [ True, False],
       [False,  True],
       [Fal

In [24]:
from sklearn.metrics import classification_report

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.85      0.69      0.76       258
           1       0.73      0.87      0.79       242

   micro avg       0.78      0.78      0.78       500
   macro avg       0.79      0.78      0.78       500
weighted avg       0.79      0.78      0.78       500
 samples avg       0.78      0.78      0.78       500



In [25]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.

In [122]:
import cv2
import imageio
def video_show(single_video):
    # Video yolu
    video_path = single_video

    # Video yükleyici
    cap = cv2.VideoCapture(video_path)

    # GIF oluşturucu
    gif_frames = []

    # Video frame'lerini okuma ve GIF'e ekleme
    while True:
        # Frame okuma
        ret, frame = cap.read()

        # Eğer frame okunamadıysa döngüden çık
        if not ret:
            break

        # Frame'i BGR'den RGB'ye dönüştürme
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Frame'i GIF'e ekleme
        gif_frames.append(frame_rgb)

    # Kaynakları serbest bırakma
    cap.release()

    # GIF dosyasını oluşturma
    imageio.mimsave("video.gif", gif_frames, fps=30)

def predict(frames):
    frames = [frames] 
    frames = np.array(frames)
    y_pred = model.predict(frames)

    # tahmin sonuçlarını kontrol edin
   
    predictions = y_pred > 0.5
    print(predictions)  # (num_samples, num_classes)


In [125]:
video_show(videos[1200])

In [126]:
predict(x_test[15])

1/1 [==============================] - 0s 72ms/step
[[False  True]]
